In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset



fname = 'results2'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,Precision,Recall,F1\n')

# SOPD.csv
df = pd.read_csv('SOPD.csv', encoding='utf-8', engine='python', sep='\t')



df.fillna('', inplace=True)  
display(len(df))


display(df.columns)
display(len(df))
display(df[:4])



classes = set(df['future_sentiment'].values)
display(classes)

c = df['future_sentiment'].value_counts()
display(c)

df['future_sentiment'] = df['future_sentiment'].astype('category')
df['label'] = df['future_sentiment'].cat.codes

df = df[['text', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


max_sequence_length = 128

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel








class CapsuleLayer(nn.Module):
   
    def __init__(self, in_dim, num_capsules, dim_capsule):
        super().__init__()
        self.in_dim = in_dim
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.linear = nn.Linear(in_dim, num_capsules * dim_capsule)

    def squash(self, s, dim=-1):
        norm = torch.norm(s, p=2, dim=dim, keepdim=True)
        scale = (norm**2) / (1 + norm**2)
        direction = s / (norm + 1e-8)
        return scale * direction

    def forward(self, x):
        x = self.linear(x) 
        x = x.view(x.size(0), self.num_capsules, self.dim_capsule)
        x = self.squash(x, dim=-1) 
        return x

class BERT_GRU_CapsNet(nn.Module):
  
    def __init__(self, 
                 model_name='faisalq/SaudiBERT',
                 num_labels=3,
                 gru_hidden_dim=128,
                 gru_num_layers=1,
                 capsule_num=10,
                 capsule_dim=16,
                 dropout_prob=0.1,
                 bidirectional=True):
       
        super().__init__()
        self.num_labels = num_labels
        self.bidirectional = bidirectional

        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size  # 768

     
        self.gru = nn.GRU(
            input_size=hidden_size,
            hidden_size=gru_hidden_dim,
            num_layers=gru_num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )

    
        cap_in_dim = gru_hidden_dim * 2 if bidirectional else gru_hidden_dim
        self.capsule = CapsuleLayer(
            in_dim=cap_in_dim,
            num_capsules=capsule_num,
            dim_capsule=capsule_dim
        )

        self.classifier = nn.Linear(capsule_num * capsule_dim, num_labels)
        self.dropout = nn.Dropout(dropout_prob)
        self.loss_fct = nn.CrossEntropyLoss()

    def forward(self, 
                input_ids=None, 
                attention_mask=None, 
                token_type_ids=None, 
                labels=None):
      
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True
        )

        sequence_output = outputs.last_hidden_state  
        gru_output, h_n = self.gru(sequence_output)
      
        if self.bidirectional:          
            final_gru_hidden = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
        else:
            final_gru_hidden = h_n[-1]

        caps_out = self.capsule(final_gru_hidden)  
        caps_flat = caps_out.view(caps_out.size(0), -1) 
        caps_flat = self.dropout(caps_flat)
        logits = self.classifier(caps_flat)

        loss = None
        if labels is not None:
            loss = self.loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )





models = [ 
        # 'aubmindlab/bert-base-arabertv02',
        # 'asafaya/bert-base-arabic',
        # 'CAMeL-Lab/bert-base-arabic-camelbert-da',
        # 'qarib/bert-base-qarib', 
        # 'UBC-NLP/ARBERTv2',
        # 'UBC-NLP/MARBERTv2',
        # 'faisalq/SaudiBERT',    
        'SaudiBERT_GRU_CapsNet',

]


seeds = [0, 1, 42, 100]

for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            if model_name == 'SaudiBERT_GRU_CapsNet':
                tokenizer = AutoTokenizer.from_pretrained('faisalq/SaudiBERT')            
                model = BERT_GRU_CapsNet(model_name='faisalq/SaudiBERT', 
                          )
                
            else:
                tokenizer = AutoTokenizer.from_pretrained(model_name)
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=classes_num).to('cuda')

                                                         
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['text'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True) 
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)  
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')  
                precision = precision_score(labels, predictions, average='macro')
                recall = recall_score(labels, predictions, average='macro')
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{precision},{recall},{f1}\n')
                return {'accuracy': acc, 'precision': precision, 'recall':recall, 'f1_score': f1}
    
    
            
            
            epochs = 8
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 50, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 50
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)
best_results = results.groupby('Model', as_index=False)['F1'].max()
best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'Precision', 'Recall', 'F1']].drop_duplicates()
best_results.to_csv(f'{fname}.csv')
print(best_results)


16152

Index(['text', 'future_sentiment', 'class'], dtype='object')

16152

,text,future_sentiment,class
0,إن شاء الله خسارة للمتغطرس مدرب اليونايتد,pessimistic,sport
1,إن شاء الله عمان تغادر البطولة,optimistic,sport
2,اتمنى استغلال السمعة الاوروبية عن فيغا وتصريفه...,optimistic,sport
3,اتوقع انهيار الخصم الان,pessimistic,sport


{'neutral', 'optimistic', 'pessimistic'}

...

Some weights of BertModel were not initialized from the model checkpoint at faisalq/SaudiBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/12921 [00:00<?, ? examples/s]

Map:   0%|          | 0/3231 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
50,0.790800,0.565928,0.835036,0.841181,0.815798,0.824685
100,0.528000,0.468530,0.872176,0.878169,0.858213,0.865514
150,0.455000,0.410147,0.892603,0.888246,0.887574,0.887845
200,0.423100,0.413263,0.882080,0.880120,0.886009,0.881093
250,0.312700,0.373614,0.898174,0.892172,0.891752,0.891928
300,0.289100,0.365776,0.900959,0.901044,0.896873,0.897468
350,0.282700,0.391304,0.887960,0.890445,0.879672,0.881166
400,0.292400,0.350549,0.900340,0.904016,0.887988,0.893139
450,0.226000,0.333677,0.904364,0.903175,0.893994,0.897836
500,0.220900,0.320492,0.907768,0.904524,0.905206,0.904589


                   Model  Accuracy  Precision   Recall        F1
0  SaudiBERT_GRU_CapsNet  0.935624   0.933818  0.93316  0.933361
